In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import numpy
import pandas
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping

#from keras.models import Sequential
from tensorflow.keras.layers import Dense ,Dropout,BatchNormalization
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [2]:
df=pd.read_csv("/notebooks/notebooks/Admission_data.csv")
#changing names because previous names are little bit confusing
df=df.rename(index=str, columns={"GRE Score": "GRE", "TOEFL Score": "TOEFL", "Chance of Admit ": "Admission_Chance"})
#we donot need serial number so its good to drop it because its just a number
df=df.drop("Serial No.",axis=1)
df.head(10)

,GRE,TOEFL,University Rating,SOP,LOR,CGPA,Research,Admission_Chance
0,337,118,4,4.5,4.5,9.65,1,0.92
1,324,107,4,4.0,4.5,8.87,1,0.76
2,316,104,3,3.0,3.5,8.00,1,0.72
3,322,110,3,3.5,2.5,8.67,1,0.80
4,314,103,2,2.0,3.0,8.21,0,0.65
5,330,115,5,4.5,3.0,9.34,1,0.90
6,321,109,3,3.0,4.0,8.20,1,0.75
7,308,101,2,3.0,4.0,7.90,0,0.68
8,302,102,1,2.0,1.5,8.00,0,0.50
9,323,108,3,3.5,3.0,8.60,0,0.45


In [3]:
admit=np.asarray(df["Admission_Chance"])
len(np.unique(admit))
#we have 60 different values in the coloum [chance to predict]

61

In [4]:
X=np.asarray(df.drop("Admission_Chance",axis=1))
Y=np.asarray(df["Admission_Chance"])

X_train, X_test, y_train, y_test = train_test_split(
     X,Y, test_size=0.2, random_state=0)
from sklearn.preprocessing import MinMaxScaler
scaler =  MinMaxScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.fit_transform(X_test)

In [5]:
model_keras = tf.keras.models.Sequential(
    [
        tf.keras.layers.Dense(16, input_dim=7, activation='relu'),
        tf.keras.layers.Dense(8, input_dim=7, activation='relu'),
        tf.keras.layers.Dense(1)  
    ]
)
model_keras.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 16)                128       
                                                                 
 dense_1 (Dense)             (None, 8)                 136       
                                                                 
 dense_2 (Dense)             (None, 1)                 9         
                                                                 
Total params: 273
Trainable params: 273
Non-trainable params: 0
_________________________________________________________________


In [6]:
# Compile model
model_keras.compile(
    optimizer='adam',
    #loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
    loss=tf.keras.losses.MeanAbsolutePercentageError(),
    metrics=["accuracy"]
)

In [7]:
# Train model
model_keras.fit(X_train, y_train, batch_size=5, epochs=20, validation_split=0.2,
          callbacks=[EarlyStopping(monitor='val_loss', patience=4)])      

Epoch 1/20
64/64 [==============================] - 1s 6ms/step - loss: 19.5927 - accuracy: 0.0000e+00 - val_loss: 16.8513 - val_accuracy: 0.0000e+00
Epoch 2/20
64/64 [==============================] - 0s 3ms/step - loss: 13.9202 - accuracy: 0.0000e+00 - val_loss: 13.5756 - val_accuracy: 0.0000e+00
Epoch 3/20
64/64 [==============================] - 0s 3ms/step - loss: 11.3385 - accuracy: 0.0000e+00 - val_loss: 11.1938 - val_accuracy: 0.0000e+00
Epoch 4/20
64/64 [==============================] - 0s 3ms/step - loss: 9.9388 - accuracy: 0.0000e+00 - val_loss: 11.2345 - val_accuracy: 0.0000e+00
Epoch 5/20
64/64 [==============================] - 0s 3ms/step - loss: 9.1329 - accuracy: 0.0000e+00 - val_loss: 9.5387 - val_accuracy: 0.0000e+00
Epoch 6/20
64/64 [==============================] - 0s 3ms/step - loss: 8.4438 - accuracy: 0.0000e+00 - val_loss: 9.2778 - val_accuracy: 0.0000e+00
Epoch 7/20
64/64 [==============================] - 0s 3ms/step - loss: 8.3122 - accuracy: 0.0000e+00 - v

In [8]:
# Test model
test_scores = model_keras.evaluate(X_test, y_test, verbose=2)
print("Test loss: ", test_scores[0])
print("Test accuracy: ", test_scores[1])

4/4 - 0s - loss: 10.2722 - accuracy: 0.0000e+00 - 177ms/epoch - 44ms/step
Test loss:  10.27215576171875
Test accuracy:  0.0


In [9]:
# define base model
'''
def baseline_model():
    # create model
    model = Sequential()
    
    
    
    model.add(Dense(16, input_dim=7, activation='relu'))
    
    
    model.add(Dense(8, input_dim=7, activation='relu'))
    
    
    model.add(Dense(1))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model
'''

"\ndef baseline_model():\n    # create model\n    model = Sequential()\n    \n    \n    \n    model.add(Dense(16, input_dim=7, activation='relu'))\n    \n    \n    model.add(Dense(8, input_dim=7, activation='relu'))\n    \n    \n    model.add(Dense(1))\n    # Compile model\n    model.compile(loss='mean_squared_error', optimizer='adam')\n    return model\n"

In [10]:
#estimator = KerasRegressor(build_fn=baseline_model, epochs=30, batch_size=3, verbose=1)

In [11]:
#estimator.fit(X_train,y_train)

In [12]:
from sklearn.metrics import accuracy_score
#prediction = model_kr.predict(X_test)
prediction = model_keras.predict(X_test)

4/4 [==============================] - 0s 4ms/step


In [13]:
train_error =  np.abs(y_test - prediction)
mean_error = np.mean(train_error)
min_error = np.min(train_error)
max_error = np.max(train_error)
std_error = np.std(train_error)

In [14]:
print("std_error: ",std_error)
print("mean_error: ",mean_error)
print("min_error: ",min_error)
print("max_error: ",max_error)

std_error:  0.11175566077630511
mean_error:  0.15698066110038758
min_error:  1.588821411135921e-05
max_error:  0.5486811017990112


In [15]:
#Model function
def model(algorithm,dtrain_X,dtrain_Y,dtest_X,dtest_Y,cols=None):

    #algorithm.fit(dtrain_X,dtrain_Y)
    predictions = algorithm.predict(dtest_X)
    prediction_probabilities = algorithm.predict(dtest_X)
    print (algorithm)
    
    return predictions,prediction_probabilities,algorithm

In [16]:
#y_pred,y_prob,model_obj=model(estimator,X_train,y_train,X_test,y_test)
y_pred,y_prob,model_obj=model(model_keras,X_train,y_train,X_test,y_test)

4/4 [==============================] - 0s 2ms/step


In [17]:
from mosaicml import *
from mosaicml.constants import MLModelFlavours

/tmp/pip_packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [18]:
@scoring_func
def score(model, request):
    payload = request.json["payload"]
    data = pd.DataFrame(eval(payload))
    prediction = model.predict(data)
    #prediction = pd.Series(model.predict(data))
    print (prediction)
    #return prediction
    return prediction.tolist()

In [19]:
cols = ['GRE Score','TOEFL Score','University Rating','SOP','LOR','CGPA','Research']

In [20]:
X_temp = pd.DataFrame(X,columns=cols)

In [21]:
import requests
req = requests.Request()
req.json = {"payload":X_temp.head(1).to_json(orient = 'records')}

In [22]:
req.json['payload']

'[{"GRE Score":337.0,"TOEFL Score":118.0,"University Rating":4.0,"SOP":4.5,"LOR":4.5,"CGPA":9.65,"Research":1.0}]'

In [23]:
req.json

{'payload': '[{"GRE Score":337.0,"TOEFL Score":118.0,"University Rating":4.0,"SOP":4.5,"LOR":4.5,"CGPA":9.65,"Research":1.0}]'}

In [24]:
#sch = generate_schema(score,(model_obj, req),X_temp)
sch = generate_schema(score,(model_keras, req),X_temp)
print(sch)

1/1 [==============================] - 0s 73ms/step
[[61.5098]]
{'input': {'type': 'pd.core.frame.DataFrame', 'items': {'type': 'object', 'properties': {'GRE Score': {'type': 'number', 'format': 'double'}, 'TOEFL Score': {'type': 'number', 'format': 'double'}, 'University Rating': {'type': 'number', 'format': 'double'}, 'SOP': {'type': 'number', 'format': 'double'}, 'LOR': {'type': 'number', 'format': 'double'}, 'CGPA': {'type': 'number', 'format': 'double'}, 'Research': {'type': 'number', 'format': 'double'}}}, 'example': [{'GRE Score': 337.0, 'TOEFL Score': 118.0, 'University Rating': 4.0, 'SOP': 4.5, 'LOR': 4.5, 'CGPA': 9.65, 'Research': 1.0}, {'GRE Score': 324.0, 'TOEFL Score': 107.0, 'University Rating': 4.0, 'SOP': 4.0, 'LOR': 4.5, 'CGPA': 8.87, 'Research': 1.0}]}, 'output': {'type': 'array', 'items': {'type': 'object'}, 'example': [[61.50979995727539]]}}


In [25]:
register_model(model_keras, score, name="", 
                description="Identify chances of Admission",
                flavour=MLModelFlavours.keras)

'name cannot be empty, blank or whitespace'